# SD-TSIA-211: Optimization for Machine Learning TP1

### Authors: Gabriele Lorenzo, Aldo Pietromatera

#### Setup:

In [23]:
import numpy as np

#### Importing data:

In [24]:
# Loading data
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load(
    "./data/data_center_data_matrix.npy", allow_pickle=True
)

# Constructing matrices for min_w ||A w - b||_2**2
matrix_mean = np.mean(data_matrix_train, axis=0)
M = data_matrix_train - matrix_mean
matrix_std = np.std(M, axis=0)
M = M / matrix_std

A = np.hstack([M, np.ones((M.shape[0], 1)), -(M.T * COP_train[:, 3]).T])
b = COP_train[:, 3]

# Constructing matrices for the test set
M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack(
    [M_test, np.ones((M_test.shape[0], 1)), -(M_test.T * COP_test[:, 3]).T]
)
b_test = COP_test[:, 3]

## Part 3: Least squares

In order to fit this model to the data, we first standardize the data. Indeed, the measurements may have various units (like kWh, degree Celsius, V, etc) and it makes more sense to separate the statistical aspects from these dimensionality considerations. Hence, we consider the matrix $\tilde{x}$, which is such that each of its columns has mean and standard deviation respectively 0 and 1 over the training set. (Note that we do not use the test set to do this
standardization.)
We then solve the following least squares problem:

$$\min\limits_w = \frac{1}{2}||Aw - b||^2$$

where $(Aw)_t = \tilde{x}(t)^T w_1 + w_0 - y(t) \times \tilde{x}(t)^T w_2$ for all $t$ and $b = y(t)$.

#### Q.3.1: Show that if $Aw = b$, then $y(t) = \frac{w_1^T \tilde{x}(t) + w_0}{w_2^T \tilde{x}(t) + 1}$.

We know that the $t^{th}$ element of $Aw$, as well as the $t^{th}$ element of $b$ are given by:

$$
\left\{\begin{matrix}
    (Aw)_t = \tilde{x}(t)^T w_1 + w_0 - y(t) \times \tilde{x}(t)^T w_2
    \\ 
    b_{t} = y(t)
\end{matrix}\right.
$$

Then, it follows that, for every $t$:

$$
    (Aw)_{t} = b_{t} \iff
    \tilde{x}(t)^{T} w_{1} + w_{0} - y(t) \times \tilde{x}(t)^{T} w_{2} = y(t) \iff
    \tilde{x}(t)^{T} w_{1} + w_{0} = y(t) \times \tilde{x}(t)^{T} w_{2} + y(t) \iff
$$

$$
    y(t) = \frac{\tilde{x}(t)^{T} w_{1} + w_{0}}{\tilde{x}(t)^{T} w_{2} + 1} \iff
    y(t) = \frac{w_{1}^{T} \tilde{x}(t) + w_{0}}{w_{2}^{T} \tilde{x}(t) + 1}
$$

#### Q.3.2: Solve this least squares problem using the function numpy.linalg.lstsq.

In [25]:
model = np.linalg.lstsq(A, b, rcond=None)
w = model[0]

print("w = ", w)

w =  [-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]


#### Q.3.3: Evaluate the quality of the solution found on the test set.

In [26]:
b_pred = A_test @ w
error = (0.5 * np.linalg.norm(b_pred - b_test) ** 2) / b_test.shape[0]
print(f"Error: {error}")

Error: 390.44923967619826


#### Q.3.4: In order to improve the generalization power of the model, we consider a $l_2$ regularization:

$$ \min\limits_w = \frac{1}{2}||Aw - b||^2 + \frac{\lambda}{2}||w||^2$$

where $\lambda = 100$. Solve this problem and compare the test mean square error with the unregularized one.

In [27]:
lambda_reg = 100
w_reg = np.linalg.solve(A.T @ A + lambda_reg * np.eye(A.shape[1]), A.T @ b)

b_pred_reg = A_test @ w_reg
error_reg = (0.5 * np.linalg.norm(b_pred_reg - b_test) ** 2) / b_test.shape[0]
print(f"Error reg: {error_reg}")

if error_reg < error:
    print("Regularization improved the generalization power of the model.")
else:
    print("Regularization did not improve the generalization power of the model.")

Error reg: 150.52741404709445
Regularization improved the generalization power of the model.


#### Q.3.5: Calculate the gradient of $f_1 : w \mapsto \frac{1}{2}||Aw - b||^2 + \frac{\lambda}{2}||w||^2$. Is the function convex?

We can use the partial derivatives of the function $f_1$ to calculate the gradient. In the end we obtain the following expression:

$$\nabla f_1(w) = A^T(Aw-b) + \lambda w$$

To prove that the function is convex we can calculate the Hessian matrix and check if it is positive semi-definite. The Hessian matrix is the following:

$$H = A^TA + \lambda I$$

Since $A^TA$ is positive semi-definite and $\lambda > 0$, the Hessian matrix is positive semi-definite and the function is convex.

#### Q.3.6: Implement gradient descent to minimize $f_1$. What step size are you choosing ? How many iterations are needed to get $w_k$ such that $||\nabla f(w_k)|| \leq 1$?

In [28]:
def gradient_descent(A, b, lambda_reg, n_iter, verbose=False):
    w_gd = np.random.randn(A.shape[1])
    # L = Lipschitz constant of the gradient of f1
    L = np.linalg.norm(A.T @ A)
    step_size = 1.9 / L

    for i in range(n_iter):
        grad = A.T @ (A @ w_gd - b) + lambda_reg * w_gd
        w_gd -= step_size * grad

        if i % 10000 == 0 and verbose:
            print("Step:", i, "Norm of grad:", np.linalg.norm(grad))

        if np.linalg.norm(grad) <= 1 and verbose:
            print(f"Number of iterations: {i}")
            print(f"Norm of grad: {np.linalg.norm(grad)}")
            break

    return w_gd

w_gd = gradient_descent(A, b, lambda_reg, n_iter=100000, verbose=True)

b_pred_gd = A_test @ w_gd
error_gd = (0.5 * np.linalg.norm(b_pred_gd - b_test) ** 2) / b_test.shape[0]
print(f"\nError gradient descent: {error_gd}")

Step: 0 Norm of grad: 2500173.6332380194
Step: 10000 Norm of grad: 2505.6432064027945
Step: 20000 Norm of grad: 1497.5319467260147
Step: 30000 Norm of grad: 936.1307167901575
Step: 40000 Norm of grad: 591.383379192776
Step: 50000 Norm of grad: 375.15125347571035
Step: 60000 Norm of grad: 238.48827959739617
Step: 70000 Norm of grad: 151.8047161269512
Step: 80000 Norm of grad: 96.71160113003442
Step: 90000 Norm of grad: 61.6513979863582

Error gradient descent: 201.83017688992086


#### Q.3.7: (Bonus question) Implement the conjugate gradient method. Compare the convergence rate with the previous algorithm.

In [29]:
def conjugate_gradient_descent(A, b, lambda_reg, n_iter, verbose=False):
    w_cgd = np.random.randn(A.shape[1])
    r = A.T @ (A @ w_cgd - b) + lambda_reg * w_cgd
    p = -r
    n_iter = 1000

    for i in range(n_iter):
        Ap = A.T @ (A @ p) + lambda_reg * p
        alpha = (r.T @ r) / (p.T @ Ap)
        w_cgd += alpha * p
        r_new = r + alpha * Ap
        beta = (r_new.T @ r_new) / (r.T @ r)
        p = -r_new + beta * p
        r = r_new

        if i % 100 == 0 and verbose:
            print("Step:", i, "Norm of grad:", np.linalg.norm(Ap))

        if np.linalg.norm(Ap) <= 1 and verbose:
            print(f"Number of iterations: {i}")
            print(f"Norm of grad: {np.linalg.norm(Ap)}")
            break

    return w_cgd

w_cgd = conjugate_gradient_descent(A, b, lambda_reg, n_iter=1000, verbose=True)

b_pred_cg = A_test @ w_cgd
error_cg = (0.5 * np.linalg.norm(b_pred_cg - b_test) ** 2) / b_test.shape[0]
print(f"\nError conjugate gradient: {error_cg}")

Step: 0 Norm of grad: 16432808723505.006
Step: 100 Norm of grad: 9583196686.9514
Step: 200 Norm of grad: 400217.9547585781
Step: 300 Norm of grad: 330765.7804483274
Step: 400 Norm of grad: 1195.1972734033043
Step: 500 Norm of grad: 0.7663805134167783
Number of iterations: 500
Norm of grad: 0.7663805134167783

Error conjugate gradient: 150.52741986415504


As we can see the conjugate gradient method converges way faster than the gradient descent method.

## Part 4: Regularization for a sparse model

You may have seen that at the optimum, the parameter w has many coordinates with small but nonzero values. In order to force most of them to be exactly 0 and thus, to concentrate on the really informative features, we can solve a Lasso problem, that is a least squares problem with $l_1$ regularization:

$$ \min\limits_w = \frac{1}{2}||Aw - b||^2 + \lambda||w||_1$$

#### Q.4.1: Write the objective function as $F_2 = f_2 + g_2$ where $f_2$ is differentiable and the proximal operator of $g_2$ is easy to compute. Recall the formula for $prox_{g2}$. Calculate the gradient of $f_2$.

We can write the objective function as follows:

$$F_2(w) = \frac{1}{2}||Aw-b||^2 + \lambda||w||_1$$

where $f_2(w) = \frac{1}{2}||Aw-b||^2$ and $g_2(w) = \lambda||w||_1$.

The proximal operator of $g_2$ is the soft thresholding operator:

$$prox_{g_2}(w) = S_{\lambda}(w) = sign(w)(|w| - \lambda)_+$$

The gradient of $f_2$ is the following:

$$\nabla f_2(w) = A^T(Aw-b)$$

#### Q.4.2: Code the proximal gradient method. Here, we will take $\lambda = 200$. What stopping test do you suggest?

In [30]:
def prox(x, gamma):
    return np.sign(x) * np.maximum(np.abs(x) - gamma, 0)

def proximal_gradient_descent(A, b, lambda_reg, n_iter, verbose=False):
    w_pgd = np.random.randn(A.shape[1])
    L = np.linalg.norm(A.T @ A)
    step_size = 1 / L

    for i in range(n_iter):
        grad = A.T @ (A @ w_pgd - b)
        w_pgd = prox(w_pgd - step_size * grad, step_size * lambda_reg)

        if i % 10000 == 0 and verbose:
            print("Step:", i, "Norm of grad:", np.linalg.norm(grad))

    return w_pgd

lambda_reg = 200
w_pgd = proximal_gradient_descent(A, b, lambda_reg, n_iter=100000, verbose=True)

b_pred_pgd = A_test @ w_pgd
error_pgd = (0.5 * np.linalg.norm(b_pred_pgd - b_test) ** 2) / b_test.shape[0]
print(f"\nError proximal gradient: {error_pgd}")

Step: 0 Norm of grad: 2712904.347589016
Step: 10000 Norm of grad: 2788.453168983582
Step: 20000 Norm of grad: 3013.759056243655
Step: 30000 Norm of grad: 3178.6708483240645
Step: 40000 Norm of grad: 2794.321258192863
Step: 50000 Norm of grad: 3218.2621328341806
Step: 60000 Norm of grad: 3045.7848311755574
Step: 70000 Norm of grad: 2564.9777517876264
Step: 80000 Norm of grad: 2529.0646762845463
Step: 90000 Norm of grad: 2522.5777787701036

Error proximal gradient: 0.42026689147543167


#### Q.4.3: We may try to accelerate the algorithm using line search. Compare the speed of the algorithm with fixed step size and with line search.

In [31]:
def f(A, b, lambda_reg, w):
    return 0.5 * np.linalg.norm(A @ w - b) ** 2 + lambda_reg * np.sum(np.abs(w))

def proximal_gradient_descent_ls(A, b, lambda_reg, n_iter, tol, verbose=False):
    w_pgd_ls = np.zeros(A.shape[1])
    beta = 0.5

    for i in range(n_iter):
        grad = A.T @ (A @ w_pgd_ls - b)
        gamma = 1

        while (
            f(A, b, lambda_reg, prox(w_pgd_ls - gamma * grad, lambda_reg * gamma))
            > f(A, b, lambda_reg, w_pgd_ls)
            + np.dot(grad, prox(w_pgd_ls - gamma * grad, lambda_reg * gamma) - w_pgd_ls)
            + (1 / (2 * gamma))
            * np.linalg.norm(
                w_pgd_ls - prox(w_pgd_ls - gamma * grad, lambda_reg * gamma)
            )
            ** 2
        ):
            gamma *= beta

        w_new = prox(w_pgd_ls - gamma * grad, lambda_reg * gamma)

        if np.linalg.norm(w_new - w_pgd_ls) < tol:
            break

        w_pgd_ls = w_new

        if verbose:
            print("Step:", i, "Norm of grad:", np.linalg.norm(grad))

        if np.linalg.norm(grad) <= 1 and verbose:
            print(f"Number of iterations: {i}")
            print(f"Norm of grad: {np.linalg.norm(grad)}")
            break

    return w_pgd_ls


w_pgd_ls = proximal_gradient_descent_ls(
    A, b, lambda_reg, n_iter=100000, tol=1e-5, verbose=True
)

b_pred_pgd_ls = A_test @ w_pgd_ls
error_pgd_ls = (0.5 * np.linalg.norm(b_pred_pgd_ls - b_test) ** 2) / b_test.shape[0]
print(f"\nError proximal gradient with line search: {error_pgd_ls}")

Step: 0 Norm of grad: 120927.59130301421
Step: 1 Norm of grad: 44664.486632928085
Step: 2 Norm of grad: 27796.21080906795
Step: 3 Norm of grad: 14892.04010809433
Step: 4 Norm of grad: 13365.158804783141
Step: 5 Norm of grad: 11899.630023453017
Step: 6 Norm of grad: 14252.803448831837
Step: 7 Norm of grad: 11722.092585211549
Step: 8 Norm of grad: 10974.269260455703
Step: 9 Norm of grad: 12398.901036473037
Step: 10 Norm of grad: 11588.164521058374
Step: 11 Norm of grad: 11188.497080288213
Step: 12 Norm of grad: 10778.312935126769
Step: 13 Norm of grad: 10328.220427564655
Step: 14 Norm of grad: 11434.279775766297
Step: 15 Norm of grad: 11351.413323488265
Step: 16 Norm of grad: 11273.640398194333

Error proximal gradient with line search: 7.365104290324181


## Part 5: Choice of the regularization parameter

A natural question when considering a regularized machine learning problem is: what is the best value for the regularization parameter $\rho$? Its goal is to force the model to choose less complex solutions in order to generalize better.
Hence, to evaluate the generalization performance, we are going to split our data into a training set $X_{train}$, $y_{train}$ and a validation set $X_{valid}$, $y_{valid}$.

Then, we solve the regression problem using the training set but test its performance on the validation set. Note that the
loss function for the validation set is not necessarily the mean square error (but we will keep it as the mean square error here).

Gathering everything the problem we are trying to solve is the following bilevel optimization problem:

$$\max\limits_{\rho \geq 0} \frac{1}{n_{valid}} \sum_{j=1}^{n_{valid}}f \lparen w,x_j,y_j \rparen$$

$$\hat{w}^{\lparen \rho \rparen} \in \arg \min \frac{1}{n_{train}} \sum_{i=1}^{n_{train}} f \lparen w,x_i,y_i \rparen + \rho R(w)$$

where $f \lparen w, X_i, y_i \rparen = \frac{1}{2}||Aw - b||^2$ and $R(w)$ is either $R(w)=\frac{1}{2}||w||_2^2$ or $R(w)=||w||_1$.

Since this is a complex nonconvex optimization problem, we are going to evaluate the accuracy on a grid of values for $\rho$, that is $\rho \in \lbrace\rho_0 a^k : k \in \lbrace0, 1, ..., K\rbrace\rbrace$ for given $\rho_0 > 0$,  $0 < a < 1$ and $K$. Then, we select the parameter $\hat{w}^{\lparen \rho \rparen}$ that has the smallest 0-1 loss on the validation set.

We first consider the $l_2$ regularization.

In [32]:
rho_list = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 200, 500]

best_rho = 0
best_error = np.inf

for rho in rho_list:
    w = np.linalg.solve(A.T @ A + rho * np.eye(A.shape[1]), A.T @ b)
    b_pred = A_test @ w
    error = (0.5 * np.linalg.norm(b_pred - b_test) ** 2) / b_test.shape[0]

    if error < best_error:
        best_error = error
        best_rho = rho

print(f"Best rho (with l2 reg): {best_rho}")
print(f"Best error (with l2 reg): {best_error}")

Best rho (with l2 reg): 50
Best error (with l2 reg): 142.84199447243205
